In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Chatbot/WikiQACorpus

/content/drive/MyDrive/Colab Notebooks/Chatbot/WikiQACorpus


In [3]:
from os import read
from keras.layers import Embedding, LSTM, Dropout,Dense
from keras import Input
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def doc_file(filename):
  f = open(filename, 'r')
  text = f.read()
  f.close()
  return text

train_doc = doc_file('WikiQA-train.txt')
test_doc = doc_file('WikiQA-test.txt')
dev_doc = doc_file('WikiQA-dev.txt')

train_doc[:200]

'how are glacier caves formed ?\tA partly submerged glacier cave on Perito Moreno Glacier .\t0\nhow are glacier caves formed ?\tThe ice facade is approximately 60 m high\t0\nhow are glacier caves formed ?\tIc'

In [4]:
import string

# preprocessing text
def clean_data(text):
  # prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  desc = text
  # convert to lower case
  desc = desc.lower()
  # tokenize
  desc = desc.split()
  # remove punctualtion fro each token
  desc = [w.translate(table) for w in desc]
  # remode hanging 's' and 'a'
  desc = [word for word in desc if len(word) > 1]
  # remove tokens withs numbers in them
  desc = [word for word in desc if word.isalpha()]
  return ' '.join(desc)

In [5]:
def split_q_and_a(doc):
  doc_dict = dict()
  # split doc to question and answser in one line
  for QA in doc.split('\n'):
    # pass if not have anything
    if len(QA) < 1:
      continue
    # split QA to question and answser
    var = QA.split('\t')
    question = clean_data(var[0])
    answser = clean_data(var[1])
    # if one question and answser is list() 
    if question not in doc_dict:
      doc_dict[question] = list()
    doc_dict[question].append('startseq ' + answser  + ' endseq')
  
  return doc_dict
    
train_dict = split_q_and_a(train_doc)
test_dict = split_q_and_a(test_doc)
dev_dict = split_q_and_a(dev_doc)

In [6]:
from keras.preprocessing.text import Tokenizer
vocabary_size = 2000
tokenizer = Tokenizer(num_words=vocabary_size, oov_token="<OOV>")

def data_tokenizer(data_dict):
  data = list()
  for key, val in data_dict.items():
    data.append(key)
    for i in range(len(val)):
      data.append(val[i])
  return data

tokenizer.fit_on_texts(data_tokenizer(train_dict))

In [7]:
haha =  tokenizer.texts_to_sequences(['hoang xuan back'])
pad_sequences(haha, maxlen=100)[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   1,   1, 429], dtype=int32)

In [8]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def data_to_sequence(data_dict, maxlen, batch_size):
  num = 0
  X1, X2, y  = list(), list(), list()
  while 1:
    for question, answser in data_dict.items():
        for i in range(len(answser)):
          x2y = answser[i].split()
          for j in range(1, len(x2y)):
            # input sequence of question
            x1 = tokenizer.texts_to_sequences([question])
            X1.append(pad_sequences(x1, maxlen=maxlen)[0])
            # input sequence of answser
            x2 = tokenizer.texts_to_sequences([' '.join(x2y[:j])])
            X2.append(pad_sequences(x2, maxlen=maxlen)[0])
            # ouput
            out = tokenizer.texts_to_sequences(x2y[j])
            y.append(to_categorical(out,num_classes=20000)[0])
            num+=1
          if num==batch_size:
            yield X1, X2, y
            X1, X2, y  = list(), list(), list()
            num = 0

In [ ]:
X1, X2, y = data_to_sequence(train_dict, 100, 6)

In [ ]:
X1[5]

In [ ]:
X2[5]

In [ ]:
y[5]

In [ ]:
tokenizer.word_index[]

In [ ]:
from keras.backend import dropout
from keras.layers.merge import add
from keras.models import Model
# model
input1 = Input(shape=(maxlen, ))
qe1 = Embedding(vocabary_size, 512)
qe2 = Dropout(0.5)(qe1)
qe3 = LSTM(256)(qe2)
input2 = Input(shape=(maxlen, ))
ae1 = Embedding(vocabary_size, 512)
ae2 = Dropout(0.5)(qe1)
ae3 = LSTM(256)(qe2)

decoder1 = add([ae3, qe3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocabary_size, activation='softmax')(decoder2)
model = Model(inputs=[input1, input2], outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

In [9]:
maxlen=100
epochs = 10
batch_size = 10
steps = len(train_dict)//batch_size
for i in range(epochs):
  X1, X2, y = data_to_sequence(train_dict, maxlen, batch_size)
  model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

TypeError: ignored